# Creación de ETLs con PySpark - HechoHistoriaCambios - GeografiaConDemografia

## Proceso de ETL para una dimensión.

Las llaves ID_XXXX presentes en el modelo hacen referencia a las llaves de la bodega. Por otra parte, en el proceso de ETL se van a tener en cuenta las llaves transaccionales (**ProyectoTransaccional**). La nomenclatura para utilizar es:

1.   ID_XXXX_DWH, para las llaves de la bodega.
2.   ID_XXXX_T, para las llaves transaccionales.


Proceso de Conexion e inicialización

In [1]:
# Configuración servidor base de datos transaccional
# Recuerde usar Estudiante_i como usuario y la contraseña asigana en el excel de conexión a maquina virtual como contraseña
db_user = 'Estudiante_59_202314'
db_psswd = 'aabb1122'
source_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'

dest_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/Proyecto_G3_202314'

# Driver de conexion
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [2]:
import os 
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import udf, col, length, isnan, when, count, regexp_replace
from datetime import datetime
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql.types import FloatType

In [3]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

if 'spark_context' in locals():
    spark_context.stop()

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


### Conexión y carga de datos

Se define la función para conexión y cargue de dataframes desde la base de datos origen y luego la función para guardar un dataframe en una tabla de la base de datos destino.

In [4]:
def obterner_dataframe_desde_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

def guardar_db(db_connection_string, df, tabla, db_user, db_psswd):
    df.select('*').write.format('jdbc') \
      .mode('append') \
      .option('url', db_connection_string) \
      .option('dbtable', tabla) \
      .option('user', db_user) \
      .option('password', db_psswd) \
      .option('driver', 'com.mysql.cj.jdbc.Driver') \
      .save()

### BLOQUE - HechoHistoriaCambios
Empezamos con el bloque de la dimensión <i>HechoHistoriaCambios</i>, su fuente de datos viene de la tabla transaccional <i>Aeropuertos</i>.

#### Extracción
A continuación, nos conectamos a la base de datos y extraemos la información deseada por medio de SQL, cargandola en un DataFrame PySpark, es decir en memoria. De la tabla de personas, En este paso, solo nos interesan la informacion de los aeropuertos (sigla, longitud, ancho_pista,clase,tipo,numero_vuelos_origen,fecha_construccion,fecha_vigencia), por lo cual se hace un filtro por medio del WHERE, buscando la informacion necesaria para los aeropuertos.

In [5]:
sql_aeropuertos = '''(SELECT DISTINCT sigla, nombre, longitud, ancho_pista, clase, tipo, numero_vuelos_origen, fecha_construccion, fecha_vigencia  FROM aeropuertos) AS Temp_aeropuertos'''
aeropuertos = obtener_dataframe_de_bd(source_db_connection_string, sql_aeropuertos, db_user, db_psswd)
aeropuertos.show(10)

+-----+--------------------+--------+-----------+-----+----------+--------------------+------------------+--------------+
|sigla|              nombre|longitud|ancho_pista|clase|      tipo|numero_vuelos_origen|fecha_construccion|fecha_vigencia|
+-----+--------------------+--------+-----------+-----+----------+--------------------+------------------+--------------+
|  7FO|             la isla|-71.6271|         14|   1A|   Privado|              171525|        2015-06-05|    2018-06-11|
|  7FO|             la isla|-71.6271|         15|   1A|   Privado|              571675|        2015-06-05|    2018-06-11|
|  7FO|             la isla|-71.6271|         18|   1A|   Privado|              994420|        2015-06-05|    2018-06-11|
|  7FU|        la escondida|-71.1935|         16|   1A|Fumigación|              252325|        2013-04-26|    2016-05-07|
|  7FU|        la escondida|-71.1935|         13|   1A|Fumigación|              126667|        2013-04-26|    2016-05-07|
|  7FU|        la escond

#### Transformación

1. Limpieza de Datos: En ese paso la idea es poder limpiar los datos asociados a vacios, valores negativos, y completar fechas.

In [6]:
# TRANSFORMACION

aeropuertos = aeropuertos.withColumn('numero_vuelos_origen', f.when(aeropuertos["numero_vuelos_origen"] < 0, aeropuertos["numero_vuelos_origen"] * -1).otherwise(aeropuertos["numero_vuelos_origen"]))
print('valores sin negativos')
aeropuertos.show(10)
aeropuertos = aeropuertos.withColumn('fecha_vigencia', f.when(aeropuertos["fecha_vigencia"] == '', '2023-12-31').otherwise(aeropuertos["fecha_vigencia"]))
print('fechas no vacias')
aeropuertos.show(10)

valores sin negativos
+-----+--------------------+--------+-----------+-----+----------+--------------------+------------------+--------------+
|sigla|              nombre|longitud|ancho_pista|clase|      tipo|numero_vuelos_origen|fecha_construccion|fecha_vigencia|
+-----+--------------------+--------+-----------+-----+----------+--------------------+------------------+--------------+
|  7FO|             la isla|-71.6271|         14|   1A|   Privado|              171525|        2015-06-05|    2018-06-11|
|  7FO|             la isla|-71.6271|         15|   1A|   Privado|              571675|        2015-06-05|    2018-06-11|
|  7FO|             la isla|-71.6271|         18|   1A|   Privado|              994420|        2015-06-05|    2018-06-11|
|  7FU|        la escondida|-71.1935|         16|   1A|Fumigación|              252325|        2013-04-26|    2016-05-07|
|  7FU|        la escondida|-71.1935|         13|   1A|Fumigación|              126667|        2013-04-26|    2016-05-07|
| 

2. Se debe agregar la fila de cambios como version 1

In [7]:
aeropuertos = aeropuertos.withColumn('cambios', f.lit(1))
print('se agrega la columna de cambios con V1')
aeropuertos.show(10)

se agrega la columna de cambios con V1
+-----+--------------------+--------+-----------+-----+----------+--------------------+------------------+--------------+-------+
|sigla|              nombre|longitud|ancho_pista|clase|      tipo|numero_vuelos_origen|fecha_construccion|fecha_vigencia|cambios|
+-----+--------------------+--------+-----------+-----+----------+--------------------+------------------+--------------+-------+
|  7FO|             la isla|-71.6271|         14|   1A|   Privado|              171525|        2015-06-05|    2018-06-11|      1|
|  7FO|             la isla|-71.6271|         15|   1A|   Privado|              571675|        2015-06-05|    2018-06-11|      1|
|  7FO|             la isla|-71.6271|         18|   1A|   Privado|              994420|        2015-06-05|    2018-06-11|      1|
|  7FU|        la escondida|-71.1935|         16|   1A|Fumigación|              252325|        2013-04-26|    2016-05-07|      1|
|  7FU|        la escondida|-71.1935|         13|  

3. Se deberia obtener la cantidad de cambios realizados por cada uno de las caracteristicas (longitud, ancho, clase, tipo, número de vuelos). Tener en cuenta que todos empiezan con Version 1, en caso de existir un cambio se deberia sumar este valor para saber que hubo un cambio.

In [11]:
window_spec = Window.partitionBy("sigla").orderBy("fecha_vigencia")

columns_to_check = ["longitud", "ancho_pista", "clase", "tipo", "numero_vuelos_origen"]
aeropuertos_check = aeropuertos;
for column in columns_to_check:
    aeropuertos_check = aeropuertos_check.withColumn(f"next_{column}", f.lead(column).over(window_spec))

for column in columns_to_check:
    aeropuertos_check = aeropuertos_check.withColumn(f"change_{column}", f.when(f.col(column) != f.col(f"next_{column}"), 1).otherwise(0))

aeropuertos_check = aeropuertos_check.withColumn("cambios_temp", sum(f.col(f"change_{column}") for column in columns_to_check))

aeropuertos_check = aeropuertos_check.withColumn("cambios", f.sum("cambios_temp").over(Window.partitionBy("sigla")))

columns_to_drop = [f"next_{column}" for column in columns_to_check] + [f"change_{column}" for column in columns_to_check] + ["cambios_temp"]
aeropuertos_check = aeropuertos_check.drop(*columns_to_drop)

aeropuertos_check = aeropuertos_check.withColumn('cambios', aeropuertos_check["cambios"] + 1)
# Obtener fechaInicio y fechaFinal
aeropuertos_check = aeropuertos_check.withColumn("fechaInicio", f.min("fecha_construccion").over(window_spec))
aeropuertos_check = aeropuertos_check.withColumn("fechaFinal", f.max("fecha_vigencia").over(window_spec))

# Cambiar el formato de fecha
aeropuertos_check = aeropuertos_check.withColumn("fechaInicio", f.date_format(f.col("fechaInicio"), "yyyyMMdd"))
aeropuertos_check = aeropuertos_check.withColumn("fechaFinal", f.date_format(f.col("fechaFinal"), "yyyyMMdd"))

aeropuertos_check.show(10)    

+-----+--------------------+--------+-----------+-----+----------+--------------------+------------------+--------------+-------+-----------+----------+
|sigla|              nombre|longitud|ancho_pista|clase|      tipo|numero_vuelos_origen|fecha_construccion|fecha_vigencia|cambios|fechaInicio|fechaFinal|
+-----+--------------------+--------+-----------+-----+----------+--------------------+------------------+--------------+-------+-----------+----------+
|  7FO|             la isla|-71.6271|         14|   1A|   Privado|              171525|        2015-06-05|    2018-06-11|      5|   20150605|  20180611|
|  7FO|             la isla|-71.6271|         15|   1A|   Privado|              571675|        2015-06-05|    2018-06-11|      5|   20150605|  20180611|
|  7FO|             la isla|-71.6271|         18|   1A|   Privado|              994420|        2015-06-05|    2018-06-11|      5|   20150605|  20180611|
|  7FU|        la escondida|-71.1935|         16|   1A|Fumigación|              25

4. Se debe generar un nuevo esquema para poder ser cargado.

In [12]:
hechohistoriacambios = aeropuertos_check.select("sigla", "cambios", "fechaInicio", "fechaFinal").distinct().withColumnRenamed('sigla','ID_Aeropuerto_T')
hechohistoriacambios.show(10)

+---------------+-------+-----------+----------+
|ID_Aeropuerto_T|cambios|fechaInicio|fechaFinal|
+---------------+-------+-----------+----------+
|            7FO|      5|   20150605|  20180611|
|            7FU|      4|   20130426|  20160507|
|            7FW|      4|   20090507|  20231231|
|            7FX|      5|   20131119|  20231231|
|            7FY|      5|   20130320|  20231231|
|            7FZ|      5|   20141111|  20231231|
|            7GA|      5|   20121025|  20151107|
|            7GB|      5|   20091006|  20231231|
|            7GC|      6|   20120816|  20231231|
|            7GD|      6|   20130516|  20231231|
+---------------+-------+-----------+----------+
only showing top 10 rows



#### Carga
Una vez realizado esto, se guardan los resultados en la base de datos destino

In [13]:
# CARGUE
guardar_db(dest_db_connection_string, hechohistoriacambios,'Proyecto_G3_202314.HechoHistoriaCambios', db_user, db_psswd)
print(hechohistoriacambios.count())

598


Verifique los resultados usando MySQL Workbench

### BLOQUE GeografiaConDemografia

La idea de este bloque es poder formar la informacion de <i>GeografiaConDemografia</i> con los datos de las tablas de <i>Aeropuertos, Divipola, PIB, Proyecciones</i>. Para esto debemos sacar la informacion necesaria de cada tabla.

#### Extracción

In [66]:
#EXTRACCION

sql_divipola = '''(SELECT CodigoDepartamento, NombreDepartamento, CodigoMunicipio, NombreMunicipio, CodigoCentroPoblado, NombreCentroPoblado, Longitud, Latitud FROM divipola) as Temp_divipola'''
sql_proyecciones = '''(SELECT DP, DPNOM, `AÑO`, `AREA GEOGRAFICA` as 'AreaGeo', `Total Hombres` as 'totalHombres', `Total Mujeres` as 'totalMujeres', `Total` FROM proyecciones WHERE `AREA GEOGRAFICA` = 'Total') as Temp_proyecciones'''
sql_pib = '''(SELECT * from pib) as Temp_pib'''

divipola = obtener_dataframe_de_bd(source_db_connection_string, sql_divipola, db_user, db_psswd)
proyecciones = obtener_dataframe_de_bd(source_db_connection_string, sql_proyecciones, db_user, db_psswd)
pib = obtener_dataframe_de_bd(source_db_connection_string, sql_pib, db_user, db_psswd)

divipola.show(5)

proyecciones.show(5)

pib.show(5)

print('divipola - proyecciones - pib')
print(divipola.columns, proyecciones.columns, pib.columns)

+------------------+------------------+---------------+---------------+-------------------+--------------------+--------------+--------------+
|CodigoDepartamento|NombreDepartamento|CodigoMunicipio|NombreMunicipio|CodigoCentroPoblado| NombreCentroPoblado|      Longitud|       Latitud|
+------------------+------------------+---------------+---------------+-------------------+--------------------+--------------+--------------+
|                91|          amazonas|          91001|        LETICIA|           91001000|             LETICIA|-69.9414267076|-4.19983693453|
|                91|          amazonas|          91001|        LETICIA|           91001017|COMUNIDAD INDÍGEN...|-69.9588125704|-4.19469878932|
|                91|          amazonas|          91001|        LETICIA|           91001016|COMUNIDAD INDÍGEN...|-69.9753934295|-4.17750342085|
|                91|          amazonas|          91001|        LETICIA|           91001030|   BARRIO SAN MIGUEL|-69.9288659026|-4.17628953753|

#### Transformación

1. Se debe estandarizar los nombres de los departamentos

In [67]:
departamentos_unicos = divipola.select("CodigoDepartamento", "NombreDepartamento").distinct()

# Muestra el resultado
departamentos_unicos.show(5)

+------------------+------------------+
|CodigoDepartamento|NombreDepartamento|
+------------------+------------------+
|                99|           vichada|
|                68|         santander|
|                86|          putumayo|
|                18|           caqueta|
|                70|             sucre|
+------------------+------------------+
only showing top 5 rows



2. Se debe normalizar los nombres de la tabla de PIB

In [68]:
pib_normal = pib.join(departamentos_unicos, pib["CodigoDepartamento(DIVIPOLA)"] == departamentos_unicos["CodigoDepartamento"], 'left')

pib_normal = pib_normal.withColumn("DEPARTAMENTOS", f.coalesce(pib_normal["NombreDepartamento"], pib_normal["DEPARTAMENTOS"]))

pib_normal = pib_normal.drop("NombreDepartamento").drop("CodigoDepartamento(DIVIPOLA)").distinct().withColumnRenamed('DEPARTAMENTOS','Departamento').withColumnRenamed('CodigoDepartamento','CodDep')
pib_normal = pib_normal.where(pib_normal.CodDep.isNotNull())
pib_normal = pib_normal.orderBy('CodDep', ascending=True)
pib_normal.show()

+------------------+--------+--------+--------------------+--------------------+--------+--------+--------------------+--------+--------+--------+--------+--------+--------+------+
|      Departamento|    2006|    2007|                2008|                2010|    2011|    2012|                2013|    2014|    2015|    2016|    2017|    2009|    2005|CodDep|
+------------------+--------+--------+--------------------+--------------------+--------+--------+--------------------+--------+--------+--------+--------+--------+--------+------+
|         antioquia|10362481|11530203| 1.208473197289277E7|1.3099961280563224E7|14580906|15552451|1.6425657101736834E7|17607117|18817790|20289054|21021421|12444001| 9340469|     5|
|         atlantico| 7194985| 8299825|   8858405.054387597|   9481200.322350295|10288314|11421539|1.2434114309437651E7|13681473|14921849|15893944|16559026| 9317808| 6388791|     8|
|           bolivar| 7044660| 7956639|    8593127.45117461|   9624568.977941897|11153868|118697

3. Generar dato de area metropolitana con latitud y longitud de centro poblado

In [78]:
# Calculo de area por arreglo de coordenadas entrantes en ° al 2 

def calculate_area(coords):
    n = len(coords)
    if n < 3:
        return 0

    if coords[0] != coords[-1]:
        coords.append(coords[0])

    area = sum((coords[i][0] * coords[i + 1][1] - coords[i + 1][0] * coords[i][1]) for i in range(n)) / 2.0
    return abs(area)

# Generar Area calculada por datos de entrada
@f.udf(FloatType())
def calculate_area_udf(longitudes, latitudes):
    if not (isinstance(longitudes, list) and isinstance(latitudes, list)):
        return 0.0
    coords = list(zip(longitudes, latitudes))
    return float(calculate_area(coords))


In [79]:
diviconarea = divipola
grouped = diviconarea.groupBy("CodigoDepartamento").agg(f.collect_list("Longitud").alias("Longitudes"),
                                                     f.collect_list("Latitud").alias("Latitudes"))

areas = grouped.withColumn("areaMetropolitana", calculate_area_udf(f.col("Longitudes"), f.col("Latitudes")))

diviconarea = diviconarea.join(areas.select("CodigoDepartamento", "areaMetropolitana"), on="CodigoDepartamento", how="left")

diviconarea.show()

+------------------+------------------+---------------+---------------+-------------------+--------------------+--------------+--------------+-----------------+
|CodigoDepartamento|NombreDepartamento|CodigoMunicipio|NombreMunicipio|CodigoCentroPoblado| NombreCentroPoblado|      Longitud|       Latitud|areaMetropolitana|
+------------------+------------------+---------------+---------------+-------------------+--------------------+--------------+--------------+-----------------+
|                91|          amazonas|          91001|        LETICIA|           91001000|             LETICIA|-69.9414267076|-4.19983693453|        5.9488072|
|                91|          amazonas|          91001|        LETICIA|           91001017|COMUNIDAD INDÍGEN...|-69.9588125704|-4.19469878932|        5.9488072|
|                91|          amazonas|          91001|        LETICIA|           91001016|COMUNIDAD INDÍGEN...|-69.9753934295|-4.17750342085|        5.9488072|
|                91|          amaz

4. Se deberia sacar los datos de PIB, totalHombres, totalMujeres

In [93]:
pib_nor = pib_normal
proy = proyecciones
pib_nor = pib_nor.withColumnRenamed("CodDep", "DP")
#pib_nor.show(5)
proy = proy.withColumn("AÑO", f.col("AÑO").cast("string"))
# proy.show(5)
pib_melted = (pib_nor.withColumn("AÑO", f.lit("2005")).select("DP", "Departamento", "AÑO", f.col("2005").alias("PIB"))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2006")).select("DP", "Departamento", "AÑO", f.col("2006").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2007")).select("DP", "Departamento", "AÑO", f.col("2007").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2008")).select("DP", "Departamento", "AÑO", f.col("2008").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2009")).select("DP", "Departamento", "AÑO", f.col("2009").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2010")).select("DP", "Departamento", "AÑO", f.col("2010").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2011")).select("DP", "Departamento", "AÑO", f.col("2011").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2012")).select("DP", "Departamento", "AÑO", f.col("2012").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2013")).select("DP", "Departamento", "AÑO", f.col("2013").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2014")).select("DP", "Departamento", "AÑO", f.col("2014").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2015")).select("DP", "Departamento", "AÑO", f.col("2015").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2016")).select("DP", "Departamento", "AÑO", f.col("2016").alias("PIB")))
             .unionByName(pib_nor.withColumn("AÑO", f.lit("2017")).select("DP", "Departamento", "AÑO", f.col("2017").alias("PIB"))))
# pib_melted.show(5)
pibproy = proy.join(pib_melted, ["DP", "AÑO"], 'inner').distinct()
pibproy.show()

+---+----+------------------+-------+------------+------------+-------+------------------+-----------+
| DP| AÑO|             DPNOM|AreaGeo|totalHombres|totalMujeres|  Total|      Departamento|        PIB|
+---+----+------------------+-------+------------+------------+-------+------------------+-----------+
|  8|2005|         atlantico|  Total|     1024962|     1089174|2114136|         atlantico|  6388791.0|
| 44|2005|        la guajira|  Total|      312535|      333183| 645718|        la guajira|  6503249.0|
| 47|2005|         magdalena|  Total|      547902|      549849|1097751|         magdalena|  3938108.0|
| 54|2005|norte de santander|  Total|      637108|      660541|1297649|norte de santander|  3928796.0|
| 95|2005|          guaviare|  Total|       40593|       32703|  73296|          guaviare|  4759905.0|
| 18|2005|           caqueta|  Total|      194127|      185146| 379273|           caqueta|  3306217.0|
| 99|2005|           vichada|  Total|       42508|       35136|  77644|  

5. calcular la fecha inicial y fecha final

In [99]:
pibproyf = pibproy.withColumn("fechaInicial", f.concat(f.col("AÑO"), f.lit("0101")))
pibproyf = pibproyf.withColumn("fechaFinal", f.concat(f.col("AÑO"), f.lit("1231")))
pibproyf = pibproyf.orderBy('AÑO', ascending=True)
pibproyf.show(5)

+---+----+------------------+-------+------------+------------+-------+------------------+---------+------------+----------+
| DP| AÑO|             DPNOM|AreaGeo|totalHombres|totalMujeres|  Total|      Departamento|      PIB|fechaInicial|fechaFinal|
+---+----+------------------+-------+------------+------------+-------+------------------+---------+------------+----------+
|  8|2005|         atlantico|  Total|     1024962|     1089174|2114136|         atlantico|6388791.0|    20050101|  20051231|
| 44|2005|        la guajira|  Total|      312535|      333183| 645718|        la guajira|6503249.0|    20050101|  20051231|
| 47|2005|         magdalena|  Total|      547902|      549849|1097751|         magdalena|3938108.0|    20050101|  20051231|
| 54|2005|norte de santander|  Total|      637108|      660541|1297649|norte de santander|3928796.0|    20050101|  20051231|
| 95|2005|          guaviare|  Total|       40593|       32703|  73296|          guaviare|4759905.0|    20050101|  20051231|


6. Organizar la informacion, agregar iterador idMunicipio_DWH y añadir version como 1

In [102]:
merged_df = pibproyf.join(
    diviconarea,
    (pibproyf.DP == diviconarea.CodigoDepartamento) & (pibproyf.Departamento == diviconarea.NombreDepartamento),
    "inner"
)

merged_df = merged_df.orderBy('AÑO',ascending=True)

geografiaConDemografie = merged_df.select(
    (f.monotonically_increasing_id() + 1).alias("idMunicipio_DWH"),
    f.col("CodigoMunicipio").alias("idMunicipio_T"),
    f.col("NombreMunicipio").alias("nombreMunicipio"),
    f.col("NombreDepartamento").alias("nombreDepartamento"),
    f.col("areaMetropolitana"),
    f.col("Longitud").alias("longitud"),
    f.col("Latitud").alias("latitud"),
    f.col("PIB"),
    f.col("totalHombres"),
    f.col("totalMujeres"),
    f.col("fechaInicial"),
    f.col("fechaFinal"),
    f.lit("20231231").alias("version")  # Asumiendo que la versión es 2023 para todos los registros
)

geografiaConDemografie.show()


+---------------+-------------+---------------+------------------+-----------------+--------------+-------------+---------+------------+------------+------------+----------+--------+
|idMunicipio_DWH|idMunicipio_T|nombreMunicipio|nombreDepartamento|areaMetropolitana|      longitud|      latitud|      PIB|totalHombres|totalMujeres|fechaInicial|fechaFinal| version|
+---------------+-------------+---------------+------------------+-----------------+--------------+-------------+---------+------------+------------+------------+----------+--------+
|              1|         5483|         NARIÑO|         antioquia|       0.28783268|-75.1933688442| 5.5213764074|9340469.0|     2630787|     2830056|    20050101|  20051231|20231231|
|              2|         5145|      CARAMANTA|         antioquia|       0.28783268|-75.6073829806|5.54640695192|9340469.0|     2630787|     2830056|    20050101|  20051231|20231231|
|              3|         5145|      CARAMANTA|         antioquia|       0.28783268|-

#### Carga

In [103]:
# CARGUE
guardar_db(dest_db_connection_string, geografiaConDemografie,'Proyecto_G3_202314.GeografiaConDemografia', db_user, db_psswd)
print(geografiaConDemografie.count())

173420


Verifique los resultados usando MySQL Workbench